# Query Data with AWS Data Wrangler
* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

AWS Data Wrangler is an open-source Python package that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

In [1]:
!pip install -q awswrangler==1.2.0

In [2]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [3]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

dataset (bool) – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [4]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                     columns=['star_rating', 'product_category', 'review_body'],
                                     filters=[("product_category", "=", "Digital_Software")],
                                     dataset=True)
df_parquet_results.shape

(102084, 3)

In [5]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,5,superb program that has found every threat la...,Digital_Software
1,5,Have used for several years and am satisfied w...,Digital_Software
2,5,I have been using this program for 4 months. I...,Digital_Software
3,3,"downloaded it, but it doesn't show up on my de...",Digital_Software
4,5,easy to use. quick updates. no problems for pa...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [6]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                filters=[("product_category", "=", "Digital_Software")],
                                dataset=True,
                                chunked=True)

In [7]:
print(next(chunk_iter))

       star_rating                                        review_body  \
0                5  superb program that  has found every threat la...   
1                5  Have used for several years and am satisfied w...   
2                5  I have been using this program for 4 months. I...   
3                3  downloaded it, but it doesn't show up on my de...   
4                5  easy to use. quick updates. no problems for pa...   
...            ...                                                ...   
39912            1  While the software in the box will work for Ma...   
39913            5  The Software Download Sample is a pretty slick...   
39914            1  The Amazon Downloader doesn't work for this pr...   
39915            5  I have used used Turbotax to do my taxes for 1...   
39916            3  My biggest complaint is that e-file costs extr...   

       product_category  
0      Digital_Software  
1      Digital_Software  
2      Digital_Software  
3      Digital_Soft

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [8]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'

In [9]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [10]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database=database_name
)

CPU times: user 274 ms, sys: 26.7 ms, total: 300 ms
Wall time: 7.91 s


In [11]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,39351822,R6081K6YSOU9W,B00M76N6MO,16665562,Intuit Quicken for Mac 2015,Digital_Software,4,2,4,N,Y,Transfer from Windows based Quicken,I successfully utilized the Quicken file conve...,2015-08-28
1,US,49271891,R2JIHZAER0886E,B00M9GTL6M,83475114,Quicken Premier Personal Finance & Budgeting S...,Digital_Software,1,2,2,N,Y,Woe to ye who go here,This program is a steaming pile of horse manur...,2015-08-28
2,US,106565,R39NNHWYQ6HZ3C,B00MHZ71G2,8655796,Norton Security with Back Up,Digital_Software,1,0,3,N,N,One Star,"false sense of security, wouldn't recommend",2015-08-28
3,US,14886874,R36V41QQIEWW3K,B00GWSJA7I,235293272,Cook'n Recipe Organizer Version 11 PC [Download],Digital_Software,5,2,2,N,Y,I love it. I had a little problem with the ...,I purchased this about 3 weeks ago. I love it...,2015-08-28
4,US,17603965,R2AESTJ8LY7LD7,B00GWSJD1G,996404290,Cook'n Recipe Organizer Version 11 MAC [Download],Digital_Software,5,1,1,N,N,The program is awesome and they are continuall...,I have been using Cook'n Recipe organizer soft...,2015-08-28


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.

In [16]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

CPU times: user 244 ms, sys: 29 ms, total: 273 ms
Wall time: 6.46 s


In [17]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    21269168   RSH1OZ87OYK92  B013PURRZW      603406193   
1             US      133437  R1WFOQ3N9BO65I  B00F4CEHNK      341969535   
2             US    45765011   R3YOOS71KM5M9  B00DNHLFQA      951665344   
3             US      113118  R3R14UATT3OUFU  B004RMK5QG      395682204   
4             US    22151364   RV2W9SGDNQA2C  B00G9BNLQE      640460561   
...          ...         ...             ...         ...            ...   
4995          US    43772834  R2BDHPYJDKXU4U  B001LRQ8IY      860645179   
4996          US    19883114   R41WEWLBI32NR  B00H4OZAHI      311771395   
4997          US    14740496  R1V3E526OROIJJ  B008D7F47Q      886655228   
4998          US    25866816  R3CU78BH4XYHQ7  B004NNVE8M      126837696   
4999          US    11391552  R1LEE669UD0W9M  B00VKW9NXW      512748721   

                                          product_title     product_category  \
0                 M

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();